<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 7: SQL DDL** 
_Logical Design in SQL._

## **Learning Objectives**
### **Theory / Be able to explain ...**
- SQL data types, constraints, and triggers
- Cascading updaties and delete to prevent referential integrity vioations
- Logical design decisions to convert data models into table schema

### **Skills / Know how to ...**
- Write SQL DDL code to create, modify, and drop tables
- Identify various types of SQL DDL statements


--------
## **LESSON 7 HIGHLIGHTS**

In [ ]:
#@title Run this cell if video does not appear
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/joDkI1ttf9o" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

---
## **BIG PICTURE: The SQL Fine Print**
In the early stages of systems design we go to great pains to separate design models from technical implementation details. We see this in database design most starkly. While the relational model defines domains and codomains, it does not actualy address data types. ER diagrams can model concepts that can't actually be implemented in relational databases without a translation step. Even the rigorous laws of the normalization process come with plenty of exceptions to allow for special cases. We can define even more normal forms but at some point it just makes sense to get on with implementation. 

Now it is finally time to put all those details back in. There is a lot more to SQL than just `SELECT` queries!

SQL is both a language and a platform. The language is a place for logical design. We create tables and define all the necessary rules to keep our data clean and informative. We then use yet more SQL to populate and maintain the tables. That's all built into the SQL language and is for the most part derived from the Relational Data Model. 

The SQL platform is 100% about implementation. Where does the data reside? How is it backed up? Who has access to the data and what are they allowed to do once they get there? These things are also supported in the SQL language, *even though they are technically not part of the Relational Database Model*. 

In this lesson we will cover SQL Data Definition Language (DDL) statements needed to implement a database design. But first, we will review some of the logical design details we didn't cover in module 3. 
 

---
## **Logical Design Decisions**

In this section we review a few logical design issues with SQL code implications.

What's logical design again? Didn't we already cover that? 

Logical design is about making decisions that **may be difficult to undo afterwards.** Before we incur the technical debt $-$ note that debt here is not about money but having options $-$ we want to be very sure that we have the conceptual requirements correct. Logical design goes past that, finally spending the techncial capital we have built up through our conceptual design work.

### **Coding Conventions**
> There are only two hard things in Computer Science: cache invalidation and naming things. -- Phil Karlton

In order to minimize typo bugs and reduce the mental burden for SQL users, it is important to always follow a consistent naming convention. It should be possible to *just know* if a query has a typo bug or a given column is likely to be spelled a certain way without having to read documentation. That's where conventions come into play. 

In the old days when most SQL coders were coming from COBOL or Fortran backgrounds, most developers followed naming conventions like:
- Table names in `ALL_CAPS`
- Column names in `Initial_Caps`
- Short but descriptive names for everything, like variables in another language
- All table and columns names in the singular form

However, as the old languages died out, SQL coders began to ignore the old naming conventions in favor of a style more like what they were used to:
- Table names with `Initial_Caps` (or `no_caps_at_all`)
- Column names in `lower_case_letters`
- Longer table names, especially in big databases with *thousands* of tables
- Collective (plural) table names and scalar (singular) column names

Names are important but not all of the story here. Coding *style* matters too. By being consistent and intentional, we can actually eliminate bugs! Some examples:
- Use uppercase for SQL keywords like `SELECT` or `AS`. This makes it clear what parts of a statement are syntax versus names of things. That way the next coder will know where to look for typos. 
- Avoid long lines of text. Instead, break statements into logical chunks (e.g., clauses in a `SELECT` query), one line at a time. Use indentation to indicate that a statement continues onto the next line. 
- When choosing data types, always prefer the standard ones over vendor-specific ones *unless* there is a major performance of other benefit and you don't mind being locked into a specific vendor's products forever.

For what it's worth, there are [style guides](https://www.sqlstyle.guide/) that dictate many of these rules so we don't have to decide. Pick a set of rules and then be consistent. SQL itself doesn't care, but every other SQL coder who works with you certainly will. 

 


### **Data Types** 

The ANSI SQL standard defines a bunch of data types, grouped here into into five kinds:
- Binary 
  - `BOOLEAN`: True or False
  - `BLOB` (Binary Large Object): Images and other binary data
- Text
  - `CHAR`: Strings of text of a fixed length
  - `VARCHAR`: Strings of text of variable but limited length
  - `CLOB` (Character Large Object): Strings of text without a maximum length (may affect performance)
- Integer Numbers
  - `INTEGER`: A standard integer with a maximum value 
  - `SMALLINT`: A space-saving version of `INTEGER` with a smaller maximum value
  - `BIGINT`: integers without a maximum value (may affect performance) 
- Decimal Numbers
  - `DECIMAL`: classic fixed precision rational numbers
  - `FLOAT`: floating point numbers with a maximum precision
  - `REAL`: floating point without limited precision (may affect performance) 
- Temporal Data
  - `DATE`: Calendar dates
  - `TIME`: Time of day
  - `TIMESTAMP`: Date plus time; some DBMSes also have `DATETIME` type
  - `INTERVAL`: Used for times between temporal values

While these types are "standard" not every DBMS vendor supports them all. The following are generally safe "consensus" choices that should survive a migration from one SQL dialect to another: 
- `VARCHAR`
- `INTEGER`
- `DECIMAL`
- `DATE`
- `TIMESTAMP`

Even with these are some suprises, like that `TIMESTAMP` has special meaning in MySQL or that SQLite doesn't have a `DATE` type at all. 

**Always RTFM for the DBMS you are working with beforehand. Then bookmark the relevant sections so you can refer back when needed.**

### **Indexes and Keys**
Indexes and keys were discussed in Lesson 4. The keys decisions usually come down to:
- **Create a surrogate primary key or use existing columns?** As a general rule we prefer surrogate keys, but there may be legitimate reasons for not doing so. For example, cross tables derived from associative entities (see *Converting ERDs to Relations* in Lesson 6) will often use a composite primary key composed of the table's *foreign keys*. That's a matter of preference, however. 
- **Create indexes for selected non-key columns to allow fast lookups?** The advantage is speed but the storage costs for the indexes can be significant. So, be judicious with indexes, using them only when the cost can be justified.
- **How do we want to enforce referential integrity for the foreign keys?** This is best addressed with constraint actions, as explained in the next section. 

### **Column Constraints**

SQL constraints are of two types:
- Rules that apply to a single column value
- Rules that involve multiple columns, rows, or tables

Here we will consider the former, leaving the latter for Lesson 8. 

Single-column constraints set rules for handling the kinds of special cases we can spot on an ER model:
![Minimum Cardinalities](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L6_Min_Cardinalities.png)

- Is the column value mandatory or can we leave it blank? Is there a default value to use if the value is not specified? 
- Do values in the column have to be unique?
- Is a column "domain native" or auto-numbered like a surrogate key? 
- Is the column a primary key (with all that entails)?
- Is the column a foreign key? If so, what does it refer to? 
- What should we do if an action would violate referential integrity?

The last issue, where updating or deleting a table row would trigger a referential integrity violation, has four possible **constraint actions**:
- **Cascade Update**: if the key of the foreign entity has changed then update the column to match.
- **Cascade Delete**: if the foreign entity has been deleted, then also delete any rows with referential integrity constraints; use this for mandatory parent-child relationships.
- **Set Null**: if the foreign key column allows null values, then set the value to null if needed; use this when the relationship is optional.
- **Restrict**: throw an error; this is default rule, used when none of the above apply. 

For obvious reasons, we would prefer not to throw errors (i.e., halt a transaction) if there is some other way to protect the data. 





---
## **SQL `CREATE TABLE` (and `DROP TABLE`) Statements**

Like we did in Lesson 2, we will start a minimal table definition and add features as we go along. 

**Heads Up:** The examples given below are based on MySQL syntax. There may be minor differences if using another SQL dialect. As always, check the manual to be sure.

### **Column Names and Data Types**
The minimal syntax to create a new table in SQL is
```sql
CREATE TABLE  tablename (
  column1 datatype1,
  column2 datatype2,
  ...
);
```

- `column1`, `column2`, are the column names.
- `datatype1`, `datatype2`, ... are SQL datatypes, which can be DMBS vendor specific.
- There is no `,` after the last column. 
- Unless backticks \``like this`\` are used all names must follow requirements (no spaces,etc.) and conventions (lowercase, etc.). 

> Some database administrators always use backticks around names. However, this practice affects readability of the code and, further, can cause seemingly nonsensical errors when a code forgets the trailing backtick on a name. It's an easy mistake; just typing a quote where you meant to have a backtick can cause lots of code immediately after the quote to become invisible to the SQL interpreter. The code becomes part of the name instead of the name being part of the code. 

Here is a snippet from the baseball database with just column definitions:
```sql
CREATE TABLE `Batting` (
  `playerID` varchar(255),
  `yearID` int(11),
  `stint` int(11),
  `teamID` varchar(255),
  `lgID` varchar(255),
  `G` int(11),
  `AB` int(11),
  `R` int(11),
  ...
);
```

- Note that the data types are in lowercase. However, since most editors (but not `%%sql` magic in Colab) support text coloring to show keywords, this is not as big a deal as it used to be. 
- Also, the `int(11)` data type is an older version of the `INTEGER` data type. (In fact `INTEGER` is an alias for `int(11)`. The implementation is the same, but that may change in the future.) For forward compatibility, new databases should use the more the modern `INTEGER` data type instead. 

### **Column Contraints**
```sql 
CREATE TABLE  tablename (
  column1 datatype1 constraint1,
  column2 datatype2 constraint2,
  ...
);
```

Constraints come in four varieties:
- `NOT NULL`
- `UNIQUE`
- `DEFAULT` defaultvalue
- `AUTO_INCREMENT` (in MySQL, though other vendors use different names)

It is possible to chain constraints one after another. `NOT NULL DEFAULT 0` would prevent null values but also set a default of `0` to use instead of throwing an error. 

Once again, here is the `Batting` table from the baseball database.
```sql
CREATE TABLE `Batting` (
  `playerID` varchar(255) NOT NULL,
  `yearID` int(11) NOT NULL,
  `stint` int(11) NOT NULL DEFAULT 1,
  `teamID` varchar(255) NOT NULL,
  `lgID` varchar(255) DEFAULT NULL,
  `G` int(11) DEFAULT 0,
  `AB` int(11) DEFAULT 0,
  `R` int(11) DEFAULT 0,
  ...
);
```

### **Indexes**
We can index a column with a special index constraint, usually located after all the column definitions. 

```sql 
CREATE TABLE  tablename (
  column1 datatype1 constraint1,
  column2 datatype2 constraint2,
  column3 datatype3 constraint3,
  ...
  INDEX (column2,column3)
);
```
 
- The `INDEX` constraint can include any subset of the columns. It will index each *combination* of the column values it finds.
- There can be multiple `INDEX` constraints. 
- Some SQL dialects require indexes and constraints to be given unique names. 

### **Primary Keys**
There are actually two ways to define a primary key, depending on how many columns are involved.

**One PK Column: in the column definition**

```sql 
CREATE TABLE  tablename (
  pkcolumn datatype1 constraint1 PRIMARY KEY,
  column2 datatype2 constraint2,
  ...
);
```


**Composite PK: as a separate index constraint**
 
```sql 
CREATE TABLE  tablename (
  pkcolumn1 datatype1 constraint1,
  pkcolumn2 datatype2 constraint2,
  column3 datatype3 constraint3,
  ...
  PRIMARY KEY (pkcolumn1,pkcolumn2)
);
```

**This syntax also works, of course, if there is only one PK column.**

Just to be different, SQLite has a special column definition syntax just for surrogate keys:
```
pkcolumn INTEGER PRIMARY KEY
```

This creates the surrogate key as an alias of the automatically-generated `rowid` index column.

Here's is the `Batting` table again:

```sql
CREATE TABLE `Batting` (
  `playerID` varchar(255) NOT NULL,
  `yearID` int(11) NOT NULL,
  `stint` int(11) NOT NULL DEFAULT 1,
  `teamID` varchar(255) NOT NULL,
  ...
  PRIMARY KEY (`playerID`,`yearID`,`stint`, `teamID`)
);
``` 

### **Foreign Keys**
Foreign keys work a lot like indexes and primary keys, except there are special rules for handling referential integrity. 

```sql
CREATE TABLE tablename (
  ...
  fkcolumn fkdatatype fkconstraint,
  FOREIGN KEY (fkcolumn) REFERENCES ftable (fpk)
    ON DELETE delete_action
    ON UPDATE update_action
```

- Foreign key columns like `fkcolumn` must be defined before the `FOREIGN KEY` constraint.
- If a foreign key is a composite, then define each of the columns and then list them all inside the constraint
- `ftable` is the name of the foreign table
- `fpk` is the primary key of the foreign table; composite primary keys can be used.
- `ON DELETE` and `ON UPDATE` are both optional; if included the `update_action` and `delete_action` are always one of `CASCADE`,`SET NULL`, or `RESTRICT` (throw an error). Almost always we want `CASCADE`. 

Here is the `Batting` table again.

```sql
CREATE TABLE `Batting` (
  `playerID` varchar(255) NOT NULL,
  `yearID` int(11) NOT NULL,
  `stint` int(11) NOT NULL DEFAULT 1,
  `teamID` varchar(255) NOT NULL,
  ...
  PRIMARY KEY (`playerID`,`yearID`,`stint`,`teamID`)
  FOREIGN KEY (`playerID) REFERENCES `Master` (`playerID`)
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
```

### **Removing a table with `DROP TABLE`**

Dropping a table will remove all traces of a table from the database. The syntax is:
```sql
DROP TABLE tablename;
```

This will throw an error if the table doesn't exist. So, most of the time we use this syntax instead: 

```sql
DROP TABLE IF EXISTS tablename;
```

When would we ever need to drop a table? 
- To remove temporary tables intended to be used just for the duration of some complex database operation.
- When reloading a database from a backup, we will want to clear out old table definitions. 
- While in the earliest drafts of our code, when the table definitions are in flux, it's often easier to recreate a table from scratch (by rerunning the code) than to alter a table in place. 






















---
## **SQL `ALTER TABLE` Statements**

It is possible to modify a table after it has been created, provided the changes don't violate an integrity constraint. Possibilities include:
- Adding new columns
- Modifying column names, datatypes, constraints
- Removing a column
- Adding or dropping indexes (primary keys, foreign keys, etc.) 
- Adding or dropping other kinds of table constraints

Given the similarities to `CREATE TABLE` statements, the following is given without further comment. 

### **Adding a New Column**

```sql
ALTER TABLE tablename 
  ADD COLUMN columnname datatype constraints;
```

### **Modifying a Column Data Type or Constraints**
```sql
ALTER TABLE tablename 
  MODIFY COLUMN old_columnnam new_columnname datatype constraints;
```

### **Renaming a Column**
```sql
ALTER TABLE tablename 
  CHANGE COLUMN columnname datatype constraints;
```

### **Dropping a Column**
```sql
ALTER TABLE tablename 
  DROP COLUMN columnname;
```

### **Adding an Index or Key**
```sql
ALTER TABLE tablename ADD PRIMARY KEY (pkcol);
ALTER TABLE tablename ADD INDEX (indexcolumn);
ALTER TABLE tablename 
  ADD FOREIGN KEY (fkcolumn) 
    REFERENCES ftable (fpk);
```

---
## **SQL Admin Utilities**
Some SQL DDL statements go beyond the relational model. These are typically for database administration tasks that are invisible to most users. 

### **`USE`, `CREATE DATABASE`, and `DROP DATABASE` Statements**
When working with multiple database connections, it can be useful to switch between them as needed with the `USE` statement:
```sql
USE databasename
```
The databasename is usually returned when establishing a database connection. 

If the database does not exist, then we can create one:
```sql
CREATE DATABASE databasename
```

Finally, we can drop a database if we no longer need it anymore:
```sql
DROP DATABASE databasename
```

For obvious reasons, the `CREATE DATABASE` and `DROP DATABASE` commands generally require special adminstrator permissions within the DBMS. 

### **`GRANT` and `REVOKE` Statements**
With the exception of SQLite and MS Access, every other major database system has user access controls that grant permissions for certain actions.

Permissions are granted on a per database and per table basis.

```sql
GRANT sql_actions ON databasename.tablename TO username@location IDENTIFIED BY password
```
- `sql_actions` is a list of allowed SQL statements (`SELECT`, etc.). Use `ALL` to allow all possible actions.
- The `databasename` and tablename can use `*` wildcards.
- The `username` only exists in the DBMS. it is not a user account in the OS. 
- The `location` is usually `localhost`, which means the database is on the same server (host) as the SQL client. That would generally be true for a webserver, for example. However, the `location` can also be an IP address (or wildcard) if the user is accessing the database remotely.
- The password is stored in the DBMS itself, in an encrypted format. 
- The `GRANT` statement should not be in version control. If you know what that means then you'll understand why. 

The reverse of `GRANT` is `REVOKE`.
```sql
REVOKE sql_actions ON databasename.tablename FROM username@location;
```

### **Vacuuming out Indexes and other Stale data**
When data is added to an indexed column, the database has to update its index accordingly. It's a relatively expensive but necessary operation. However, when we delete data from an indexed column, there is almost harm to just leaving the data in the index, skipping over it instead of deleting it. That avoid the cost of updating the index. 

We call ths sort of thing a **soft delete**. A soft delete marks data for deletion at a later time, when it won't delay more important and timely operations. It is often quicker to delete lots of things at once than to delete them one at a time. 

At some point however, soft deletes can make our tables like an episode of *Hoarders*, with lots of leftover artifacts to navigate around. Everything then grinds to halt. 

Vacuuming is the process of rebuilding an index (or sometimes a whole table) to eliminate all the soft deletes. For a large database, this process may take a while. This is the kind of thing a database administrator will schedule for the middle of the night when it will affect the fewest users. 

Vacuuming out stale data is implemented in various ways, depending on the vendor. However, both PostreSQL and SQLite support a one line SQL command that handles all tables in the database.

```sql
VACUUM;
```

### **Metadata Queries**
One of the convenient features of SQL is that **databases contain their own metadata about themselves**. This allows a lot of this structural information to be accessed in SQL queries. 

Unfortunately, how to access the metadata varies from vendor to vendor. For example:
- In MySQL, information about tables, columns, etc. is kept in the `INFORMATION_SCHEMA` database. The user has to be granted permissions to it like any other database. Alternatively, if we want a list of tables in the current database, we can use the MySQL-specific `SHOW TABLES` and `DESCRIBE` statements. 
- In SQLite, we can use the `pragma.table_info()` function within a `SELECT` statements to get information about the columns of a specific. To get a list of all tables, we query the `sqlite_master` table.  

### **Dump Files**
Databases systems are not like other software you may have encourntered before. They are **always** reading and writing from/to storage. Thus, making a backup is not simply a matter of shutting down the DBMS and copying some files. That would both take the applications they support offline and possibly corrupt data. 

Instead, the DBMS provides a way to export the current state of the database to disk as a *dump file*. A dump file is a (possibly very very long) list of SQL statements that recreate the database. These files an then be archived like any other file. If needed the database can be loaded (or reloaded) from a dump file. 

Since dump files are written in SQL, this gives the database administrator the ability to rewrite the SQL when migrating the data from one SQL dialect to another. So, if Oracle does things differently from Microsoft SQL Server, then the SQL syntax, data types, etc. in the dump files are adjusted accordingly. 



---
## **Movies Tonight, Part 3**

We continue the Movies Tonight case by converting the ERD and relations into SQLite DDL code. The database will reside in Google Drive so that in Lesson 8 we can continue right where we left off. Please run any code cells you find. The will do the the work but only if you run them. 

### **Conceptual Design**
![ERD from Lesson 5](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L6_MoviesTonight_v2.png)

- `Artist(`**`artistID`**, `name)`
- `Movie(`**`movieID`**, `title,rating)`
- `Theater(`**`theaterID`**, `name, location, phone)`
- `Credit(`**`showId`**, `ccode`, <u>`movieID`</u>,<u>`artistID`</u>`)`
- `Show(`**`showId`**, `showtime`, <u>`movieID`</u>,<u>`theaterID`</u>`)` 

### **System Admin Tasks**

The code below creates folder in Google Drive for our SQLite database. 











In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create the BUAN6510/data/MoviesTonight folder in Google Drive
from pathlib import Path
data_root = Path("./drive/My Drive/Colab Notebooks/BUAN6510")
if not data_root.exists():
  print(
      '''
      Warning! The folder '/Colab Notebooks/BUAN6510' could not be found in the connected Google Drive. 
      Please make 100% sure that both Colab and Chrome are set up use your @student.fairfield.edu account. 
      For now, a new folder with the correct path has been created in whatever Google Drive it found. 
      ''')
data_root = data_root / 'data' / 'MoviesTonight'
data_root.mkdir(parents=True, exist_ok=True)

This cell creates a symlink so that SQLite can find the new Google Drive folder. 

In [2]:
%%bash
ln -s drive/My\ Drive/Colab\ Notebooks/BUAN6510 buan6510

The Google Drive ready to hold our data, we now move on to setting up %%sql magic and SQLite. While we are at it, we will create the new database file. 

In [ ]:
# Load %%sql magic
%load_ext sql

# Standard Imports
import sqlite3
import pandas as pd

# Database connection
%sql sqlite:///buan6510/data/MoviesTonight/MoviesTonight.db

### **SQL Table Definitions**
With the database initialized and ready for input, we can define the tables all at once in a single cell. 

- Notice the order in which the tables are defined. The strong entities come before the weak ones. We will load data in that order as well.
- Comments are used to tell us what the code does and why.
- `DROP TABLE` statements are used to clear out everything so we can start over. The effect is that we can rerun the code whenever we like without triggering any errors. 

In [ ]:
%%sql

/* ------ Table Definitions ----------*/

-- The theaters table 

DROP TABLE IF EXISTS theaters;    -- Start over from scratch
CREATE TABLE theaters(
  theater_id INTEGER PRIMARY KEY, -- SQLites uses this syntax for surrogate keys
  name TEXT NOT NULL,             -- SQLite uses TEXT datatype for all text
  location TEXT NOT NULL,
  phone TEXT
);

-- The movies table

DROP TABLE IF EXISTS movies;
CREATE TABLE movies (
  movie_id INTEGER PRIMARY KEY,  
  title TEXT NOT NULL,             
  rating TEXT
);

-- The artists table

DROP TABLE IF EXISTS artists;
CREATE TABLE artists (
    artist_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    bio TEXT                       -- Included for future use
); 

-- The shows table

DROP TABLE IF EXISTS shows;
CREATE TABLE shows (
    show_id INTEGER PRIMARY KEY,
    showtime TEXT NOT NULL,        -- We may revisit the data type later
    movie_id INTEGER NOT NULL,
    theater_id INTEGER NOT NULL,
    FOREIGN KEY (movie_id) REFERENCES movies (movie_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE,
    FOREIGN KEY (theater_id) REFERENCES theaters (theater_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
);

-- The credits table

DROP TABLE IF EXISTS credits;
CREATE TABLE credits (
    credit_id INTEGER PRIMARY KEY,
    credit_code TEXT NOT NULL,
    movie_id INTEGER NOT NULL,
    artist_id INTEGER NOT NULL,
    FOREIGN KEY (movie_id) REFERENCES movies (movie_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE,
    FOREIGN KEY (artist_id) REFERENCES artists (artist_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
);

---
## **PRO TIPS: How to create a table from existing CSV data**



---
## **SQL AND BEYOND: SQLAlchemy and Object Relational Mapping**

If we dig into the innards of `%%sql` magic we find that it is mostly a *wrapper* for [SQLAlchemy](https://www.sqlalchemy.org/), a Python library that does all the heavy lifting of connecting to databases, sending SQL requests, and formatting SQL responses. 

For example, the following Python code connects to our MoviesTonight database and displays metadata for the tables found inside.












 







  

 








In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///buan6510/data/MoviesTonight/MoviesTonight.db')

with engine.connect() as conn:
  resultset = conn.execute("SELECT * FROM sqlite_master") 
  for row in resultset:
    print(row)

('table', 'theaters', 'theaters', 2, 'CREATE TABLE theaters(\n  theater_id INTEGER PRIMARY KEY, -- SQLites uses this syntax for surrogate keys\n  name TEXT NOT NULL,             -- SQLite uses TEXT datatype for all text\n  location TEXT NOT NULL,\n  phone TEXT\n)')
('table', 'movies', 'movies', 3, 'CREATE TABLE movies (\n  movie_id INTEGER PRIMARY KEY,  \n  title TEXT NOT NULL,             \n  rating TEXT\n)')
('table', 'artists', 'artists', 4, 'CREATE TABLE artists (\n    artist_id INTEGER PRIMARY KEY,\n    name TEXT NOT NULL,\n    bio TEXT                       -- Included for future use\n)')
('table', 'shows', 'shows', 5, 'CREATE TABLE shows (\n    show_id INTEGER PRIMARY KEY,\n    showtime TEXT NOT NULL,        -- We may revisit the data type later\n    movie_id INTEGE ... (120 characters truncated) ... CASCADE\n      ON UPDATE CASCADE,\n    FOREIGN KEY (theater_id) REFERENCES theaters (theater_id)\n      ON DELETE CASCADE\n      ON UPDATE CASCADE\n)')
('table', 'credits', 'credits

Each row is a *tuple* -- that's a Python thing -- with a table and the SQL DDL used to create it. Even the comments are there. 

> What follows is mostly for the software engineers who might be taking the class. If that's not you, then just try to follow along with the concepts in bold without worrying too much about the coding details. 

SQLAlchemy is more than just a database engine, however. It can also be used for **object relational mapping (ORM)**. ORM is a solution to the **impedance mismatch** problem in object oriented languages like Java, C#, or Python. In object oriented languages things like entities and relationships are represented by software objects. These objects are defined in similar fashion to relational entities but can include functionality (methods) as well as data. The code is written in **object classes**, which can be instantiated as **object instances** in code. (That's just like how a table represents an entity class but each row represents an entity instance.)

**So, if the programmer has to define object classes anyway, why do it yet again in SQL code? After all, that's just one more thing to have to keep track of as they are developing their code.** ORM makes it so they don't have to. Instead they create *mappings* called *schema* that translate their software objects into equivalent SQL code. 

In SQLAlchemy, the programmers extend a special `Base` class in their Python code that automatically translates (maps) from objects to tables (and back again):

```python
# Define the Movie schema
class Movie(Base):
    __tablename__ = 'movies'
    id = Column('movie_id', Integer(), primary_key=True, nullable=False),
    title = Column('title', String(100),  nullable=False),
    rating = Column('rating', String(5))

  def __repr__(self):
        return "<Movie(id='%i', title='%s', rating='%s')>" % (self.id, self.title, self.rating)

```

The programmers can then ignore SQL for the most part, creating and using their software objects as they choose. Then, when needed they can **persist** (store) objects in the database for retrieval later. SQL is generated as needed behind the scenes.  

```python
# A session object represents a series of database transactions
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

# create a new movie object 
movie = Movie(title='Big Night',rating ='PG-13')

# queue it up for insertion into the database; this generates a SQL query
session.add(movie)

# run the necessary SQL queries to sync the objects 
session.commit()

# retrieve a copy of the movie from the database
movie2 = session.query(Movie).filter_by(title = 'Big Night').first()

# calls the __repr__ method before printing
print(movie2)


---
## **Congratulations! You've made it to the end of Lesson 7.**

Now that we finally know how to create a database from scratch, all we have left to learn is how to work with data once it's in the database. That's in Lesson 8, of course. After that it's all applications of things we already know. 



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `BUAN6510` folder so you can find it next time.